In [1]:
# Step 1: If you already have an existing Pinecode DB that is unrelated to this notebook
#  and if you are on a trial version with Pinecode DB, 
# delete your existing DB (if you don't have an existing DB, don't mind this step)

# Step 2: Do you already have an existing Pinecone DB for this notebook?
has_existing_pinecone_db = True

# Step 3: have you already uploaded your embeddings to the Pinecode DB?
has_uploaded_embeddings = False

In [2]:
# Load environment variables

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
# Import OpenAI wrapper

from langchain.llms import OpenAI

In [4]:
# import schema for chat messages and ChatOpenAI in order to query chatmodels GPT-3.5-turbo or GPT-4

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [5]:
# Import law-related content

content = """REPUBLIC ACT NO. 7438
April 27, 1992
 
AN ACT DEFINING CERTAIN RIGHTS OF PERSON ARRESTED, DETAINED OR UNDER CUSTODIAL INVESTIGATION AS WELL AS THE DUTIES OF THE ARRESTING, DETAINING AND INVESTIGATING OFFICERS, AND PROVIDING PENALTIES FOR VIOLATIONS THEREOF.
 
 
 
Section 1.  Statement of Policy. - It is the policy of the Senate to value the dignity of every human being and guarantee full respect for human rights.
 
Sec. 2. Rights of Persons Arrested, Detained or under Custodial Investigation; Duties of Public Officers. - (a) Any person arrested detained or under custodial investigation shall at all times be assisted by counsel.
 
(b) Any public officer or employee, or anyone acting under his order or his place, who arrests, detains or investigates any person for the commission of an offense shall inform the latter, in a language known to and understood by him, of his rights to remain silent and to have competent and independent counsel, preferably of his own choice, who shall at all times be allowed to confer privately with the person arrested, detained or under custodial investigation. If such person cannot afford the services of his own counsel, he must be provided with a competent and independent counsel by the investigating officer.
 
(c)  The custodial investigation report shall be reduced to writing by the investigating officer, provided that before such report is signed, or thumbmarked if the person arrested or detained does not know how to read and write, it shall be read and adequately explained to him by his counsel or by the assisting counsel provided by the investigating officer in the language or dialect known to such arrested or detained person, otherwise, such investigation report shall be null and void and of no effect whatsoever.
 
(d) Any extrajudicial confession made by a person arrested, detained or under custodial investigation shall be in writing and signed by such person in the presence of his counsel or in the latter's absence, upon a valid waiver, and in the presence of any of the parents, elder brothers and sisters, his spouse, the municipal mayor, the municipal judge, district school supervisor, or priest or minister of the gospel as chosen by him; otherwise, such extrajudicial confession shall be inadmissible as evidence in any proceeding.
 
(e) Any waiver by a person arrested or detained under the provisions of Article 125 of the Revised Penal Code, or under custodial investigation, shall be in writing and signed by such person in the presence of his counsel; otherwise the waiver shall be null and void and of no effect.
 
(f) Any person arrested or detained or under custodial investigation shall be allowed visits by or conferences with any member of his immediate family, or any medical doctor or priest or religious minister chosen by him or by any member of his immediate family or by his counsel, or by any national non-governmental organization duly accredited by the Commission on Human Rights of by any international non-governmental organization duly accredited by the Office of the President. The person's "immediate family" shall include his or her spouse, fiance or fiancee, parent or child, brother or sister, grandparent or grandchild, uncle or aunt, nephew or niece, and guardian or ward.
 
As used this Act, "custodial investigation" shall include the practice of issuing an "invitation" to a person who is investigated in connection with an offense he is suspected to have committed, without prejudice to the liability of the "inviting" officer for any violation of law.
 
Sec. 3. Assisting Counsel. - Assisting counsel is any lawyer, except those directly affected by the case, those charged with conducting preliminary investigation or those charged with the prosecution of crimes.
 
The assisting counsel other than the government lawyers shall be entitled to the following fees:
 
(a) The amount of One hundred fifty pesos (P150.00) if the suspected person is chargeable with light felonies;
 
(b) The amount of Two hundred fifty pesos (P250.00) if the suspected person is chargeable with less grave of grave felonies;
 
(c)  The amount of Three hundred fifty pesos (P350.00) if the suspected person is chargeable with a capital offense.
 
The fee for the assisting counsel shall be paid by the city or municipality where the custodial investigation is conducted, provided that if the municipality of city cannot pay such fee, the province comprising such municipality or city shall pay the fee: Provided, That the Municipal of City Treasurer must certify that no funds are available to pay the fees of assisting counsel before the province pays said fees.
 
In the absence of any lawyer, no custodial investigation shall be conducted and the suspected person can only be detained by the investigating officer in accordance with the provisions of Article 125 of the Revised Penal Code.
 
Sec. 4. Penalty Clause. - (a) Any arresting public officer of employee, or any investigating officer, who fails to inform any person arrested, detained or under custodial investigation of his right to remain silent and to have competent and independent counsel preferably of his own choice, shall suffer a fine of Six thousand pesos (P6,000.00) or a penalty of imprisonment of not less than eight (8) years but not more than ten (10) years, or both. The penalty of perpetual absolute disqualification shall also be imposed upon the investigating officer who has been previously convicted of a similar offense.
The same penalties shall be imposed upon a officer or employee or anyone acting upon orders of such investigating officer or in his place, who fails to provide a competent and independent counsel to a person arrested, detained or under custodial investigation for the commission of an offense if the latter cannot afford the services of his own counsel.
 
(b) Any person who obstruct, persons or prohibits any lawyer, any member of the immediate family of a person arrested, detained or under custodial investigation, or any medical doctor or priest or religious minister chosen by him or by any member of his immediate family or by his counsel, from visiting and conferring privately with him, of from examining and treating him, or from ministering to his spiritual needs, at any hour of the day or, in urgent cases, of the night shall suffer the penalty of imprisonment of not less than four (4) years nor more than six (6) years, and a fine of four thousand pesos (P4,000.00).
The provisions of the above Section notwithstanding, any security officer with custodial responsibility over any detainee or prisoner may undertake such reasonable measures as may be necessary to secure his safety and prevent his escape.
 
Sec. 5. Repealing Clause. - Republic Act No. No. 857, as amended, is hereby repealed. Other laws, presidential decrees, executive orders or rules and regulations, or parts thereof inconsistent with the provisions of this Act are repealed or modified accordingly.
 
Sec. 6. Effectivity. - This Act shall take effect fifteen (15) days following its publication in the Official Gazette or in any daily newspapers of general circulation in the Philippines.
 
Approved: April 27, 1992"""

In [6]:
# Import utility for splitting up texts and split up the explanation given above into document chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 0,
)

texts = text_splitter.create_documents([content])

In [7]:
# Individual text chunks can be accessed with "page_content"

texts[0].page_content

'AN ACT DEFINING CERTAIN RIGHTS OF PERSON ARRESTED, DETAINED OR UNDER CUSTODIAL INVESTIGATION AS'

In [8]:

# Import and instantiate OpenAI embeddings

from langchain.embeddings import OpenAIEmbeddings

### wont' work
# embeddings = OpenAIEmbeddings(model_name="ada")

embeddings = OpenAIEmbeddings()

In [9]:
# Turn the first text chunk into a vector with the embedding

query_result = embeddings.embed_query(texts[1].page_content)
print(query_result)

[-0.015186337754130363, -0.014804976992309093, -0.003285833168774843, -0.040533218532800674, -0.04091458022594452, 0.018019303679466248, -0.011767708696424961, -0.00637758057564497, 4.4983353291172534e-05, -0.024679500609636307, 0.013626843690872192, -0.0040247198194265366, -0.01481859665364027, 0.005533138755708933, -0.008594240993261337, 0.011039037257432938, 0.018155504018068314, -0.015063757076859474, -0.0017433642642572522, -0.0028516945894807577, -0.014083114452660084, 0.02297699637711048, -0.016711780801415443, 0.035711728036403656, 0.01833256520330906, -0.019245106726884842, 0.012155880220234394, -0.04688015580177307, 0.018101025372743607, -0.008029010146856308, -0.00326370052061975, -0.008287790231406689, -0.028002789244055748, -0.027580568566918373, -0.03742785379290581, -0.0029538448434323072, -0.005182422697544098, 0.006786181591451168, 0.004627406597137451, -0.0017212317325174809, 0.03889881446957588, -0.03405008465051651, -0.020525390282273293, -0.041132502257823944, 0.01

In [10]:

# Import and initialize Pinecone client

import os
import pinecone
from langchain.vectorstores import Pinecone


pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),  
    environment=os.getenv('PINECONE_ENV')  
)

/home/thislexic/tm-learnings/temasek-delphi/langchain/venv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [11]:
# Create an index in Pinecone

index_name = "benv-learnings-law"

### if this doesn't work, create it manually
if not has_existing_pinecone_db:
    pinecone.create_index(index_name, dimension=1536,
                          metric="cosine", pods=1, pod_type="p1.x1")

In [12]:
# Upload vectors to Pinecone

### run this if you haven't already uploaded your embeddings to Pinecone

if not has_uploaded_embeddings:
    search = Pinecone.from_documents(texts, embeddings, index_name=index_name)

In [13]:
# Get the vector store (Pinecone)

### run this if you have already uploaded your embeddings to Pinecone

if has_uploaded_embeddings:
    index = pinecone.Index(index_name)
    search = Pinecone(index, embeddings.embed_query, "text")

In [21]:
# Do a simple vector similarity search

user_query = "What can the police do if I get pulled over due to a traffic violation"

query = user_query
results = search.similarity_search(query)
results

[Document(page_content='without prejudice to the liability of the "inviting" officer for any violation of law.', metadata={}),
 Document(page_content='without prejudice to the liability of the "inviting" officer for any violation of law.', metadata={}),
 Document(page_content='person can only be detained by the investigating officer in accordance with the provisions of', metadata={}),
 Document(page_content='person can only be detained by the investigating officer in accordance with the provisions of', metadata={})]

In [22]:
documents = ', '.join([result.page_content for result in results])
documents

'without prejudice to the liability of the "inviting" officer for any violation of law., without prejudice to the liability of the "inviting" officer for any violation of law., person can only be detained by the investigating officer in accordance with the provisions of, person can only be detained by the investigating officer in accordance with the provisions of'

In [23]:
# Setup ChatGPT
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)

In [24]:
system_message = f"""You are a helpful AI assistant specializing in Philippine criminal law.
To answer questions from the user, use the list of documents below.
Use language that can be understood by a 10 year old.
If the questions can't be answered from the documents, say you don't know. DON'T MAKE ANYTHING UP.

Documents
==========
{documents}
"""

messages = [
    SystemMessage(content=system_message),
    HumanMessage(content=user_query)
]

## Costs Money
response=chat(messages)
content=response.content

In [25]:
content

"If you get pulled over by the police for a traffic violation, they have the authority to do a few things. They can ask for your driver's license and vehicle registration to check if you are allowed to drive and if your vehicle is registered. They can also ask you questions about the violation and may issue a ticket or citation if they believe you have broken a traffic law. In some cases, they may even ask you to step out of the vehicle for further investigation. However, they cannot detain you for a long time without a valid reason or without following the proper procedures."